# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Pressure
City_ID,,,,,,,,,,
0,Puerto Ayora,-0.7393,-90.3518,72.46,80,73,10.42,EC,1733169497,1011
1,Fort St. John,56.2499,-120.8529,17.49,93,75,4.61,CA,1733169582,1009
2,Tiksi,71.6872,128.8694,-9.71,100,50,5.46,RU,1733169434,1022
3,Stanley,54.8680,-1.6985,36.18,86,20,6.91,GB,1733169436,1019
4,Egilsstadir,65.2653,-14.3948,13.91,67,0,6.91,IS,1733169586,1019


In [3]:
# Date Cleaning w/dates
city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Pressure
City_ID,,,,,,,,,,
0,Puerto Ayora,-0.7393,-90.3518,72.46,80,73,10.42,EC,2024-12-02 19:58:17,1011
1,Fort St. John,56.2499,-120.8529,17.49,93,75,4.61,CA,2024-12-02 19:59:42,1009
2,Tiksi,71.6872,128.8694,-9.71,100,50,5.46,RU,2024-12-02 19:57:14,1022
3,Stanley,54.8680,-1.6985,36.18,86,20,6.91,GB,2024-12-02 19:57:16,1019
4,Egilsstadir,65.2653,-14.3948,13.91,67,0,6.91,IS,2024-12-02 19:59:46,1019


In [4]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 586 entries, 0 to 585
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City        586 non-null    object        
 1   Lat         586 non-null    float64       
 2   Lng         586 non-null    float64       
 3   Max Temp    586 non-null    float64       
 4   Humidity    586 non-null    int64         
 5   Cloudiness  586 non-null    int64         
 6   Wind Speed  586 non-null    float64       
 7   Country     581 non-null    object        
 8   Date        586 non-null    datetime64[ns]
 9   Pressure    586 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 50.4+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 70
max_temp = 80
max_wind = 10

# Drop any rows with null values
ideal_w = city_data_df.dropna()

mask = (ideal_w["Max Temp"] >= min_temp) & (ideal_w["Max Temp"] < max_temp) & (ideal_w["Wind Speed"] < max_wind)
ideal_w = ideal_w.loc[mask].reset_index(drop=True)

# Display sample data
ideal_w

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Pressure
0,Kourou,5.1552,-52.6478,77.95,92,99,9.01,GF,2024-12-02 19:59:16,1010
1,Wailua Homesteads,22.0669,-159.3780,77.79,80,75,9.22,US,2024-12-02 19:58:07,1017
2,Dawei,14.0833,98.2000,71.37,79,56,3.60,MM,2024-12-02 20:00:33,1009
3,Jwaneng,-24.6004,24.7303,79.50,18,53,4.90,BW,2024-12-02 20:00:34,1007
4,Cabo San Lucas,22.8909,-109.9124,77.76,61,4,4.00,MX,2024-12-02 19:56:04,1014
...,...,...,...,...,...,...,...,...,...,...
88,Manokwari,-0.8667,134.0833,78.28,84,97,3.27,ID,2024-12-02 20:11:16,1007
89,Gorontalo,0.5412,123.0595,75.25,90,99,3.36,ID,2024-12-02 20:09:02,1007
90,Capitán Bado,-23.2667,-55.5333,77.88,84,50,9.44,PY,2024-12-02 20:11:28,1004
91,Biltine,14.5333,20.9167,72.37,15,6,8.93,TD,2024-12-02 20:11:29,1014


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
hotel_rows = []

for index, row in ideal_w.iterrows():
    # from the Weather CSV
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Set the parameters for the type of place
    categories = "accommodation.hotel"
    radius = 10000 # 10km
    
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5
    
    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()
        
        # Print the results
        results = data.get("features", [])
        
        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]
    
            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")
        
            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None
            }

    # append to hotel list
    hotel_rows.append(hotel_row)

In [8]:
hotel_rows

[{'city': 'Kourou',
  'country': 'GF',
  'latitude': 5.1552,
  'longitude': -52.6478,
  'address': 'SCI Horizon, 54 Rue du Docteur Hervé Floch, 97310 Kourou, France',
  'name': 'SCI Horizon',
  'distance': 299},
 {'city': 'Wailua Homesteads',
  'country': 'US',
  'latitude': 22.0669,
  'longitude': -159.378,
  'address': 'Hilton Garden Inn Kauai Wailua Bay, Kuhio Highway, Kauaʻi County, HI 96746, United States of America',
  'name': 'Hilton Garden Inn Kauai Wailua Bay',
  'distance': 4923},
 {'city': 'Dawei',
  'country': 'MM',
  'latitude': 14.0833,
  'longitude': 98.2,
  'address': 'Royal Dawei Hotel, Say Yone Road, Dawei, 60101, Myanmar',
  'name': 'Royal Dawei ဟိုတယ်',
  'distance': 784},
 {'city': 'Jwaneng',
  'country': 'BW',
  'latitude': -24.6004,
  'longitude': 24.7303,
  'address': 'Mokala Lodge & Teemane Casino, Kgorwe Avenue, Jwaneng, Botswana',
  'name': 'Mokala Lodge & Teemane Casino',
  'distance': 187},
 {'city': 'Cabo San Lucas',
  'country': 'MX',
  'latitude': 22.890

In [9]:
hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance
0,Kourou,GF,5.1552,-52.6478,"SCI Horizon, 54 Rue du Docteur Hervé Floch, 97...",SCI Horizon,299.0
1,Wailua Homesteads,US,22.0669,-159.3780,"Hilton Garden Inn Kauai Wailua Bay, Kuhio High...",Hilton Garden Inn Kauai Wailua Bay,4923.0
2,Dawei,MM,14.0833,98.2000,"Royal Dawei Hotel, Say Yone Road, Dawei, 60101...",Royal Dawei ဟိုတယ်,784.0
3,Jwaneng,BW,-24.6004,24.7303,"Mokala Lodge & Teemane Casino, Kgorwe Avenue, ...",Mokala Lodge & Teemane Casino,187.0
4,Cabo San Lucas,MX,22.8909,-109.9124,"Comfort Rooms, Calle Revolución de 1910, Juáre...",Comfort Rooms,65.0
...,...,...,...,...,...,...,...
88,Manokwari,ID,-0.8667,134.0833,"Hotel Arfak, Jalan Brawijaya, Manokwari 98311,...",Hotel Arfak,645.0
89,Gorontalo,ID,0.5412,123.0595,"Hotel Karawang Gorontalo, Jalan Nani Wartabone...",Hotel Karawang Gorontalo,153.0
90,Capitán Bado,PY,-23.2667,-55.5333,"Hotel Esmeralda, Avenida Internacional, Capitá...",Hotel Esmeralda,797.0
91,Biltine,TD,14.5333,20.9167,"فندق ثلاث نجوم بلتن, Abeche - Biltine National...",فندق ثلاث نجوم بلتن,612.0


In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

# Use the Pandas copy function to create a new DataFrame called hotel_df
hotel_df = ideal_w[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Kourou,GF,5.1552,-52.6478,92,
1,Wailua Homesteads,US,22.0669,-159.3780,80,
2,Dawei,MM,14.0833,98.2000,79,
3,Jwaneng,BW,-24.6004,24.7303,18,
4,Cabo San Lucas,MX,22.8909,-109.9124,61,
...,...,...,...,...,...,...
88,Manokwari,ID,-0.8667,134.0833,84,
89,Gorontalo,ID,0.5412,123.0595,90,
90,Capitán Bado,PY,-23.2667,-55.5333,84,
91,Biltine,TD,14.5333,20.9167,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Kourou - nearest hotel: Hotel Santa Eliza
Wailua Homesteads - nearest hotel: Hotel Santa Eliza
Dawei - nearest hotel: Hotel Santa Eliza
Jwaneng - nearest hotel: Hotel Santa Eliza
Cabo San Lucas - nearest hotel: Hotel Santa Eliza
Nieuw Amsterdam - nearest hotel: Hotel Santa Eliza
Owando - nearest hotel: Hotel Santa Eliza
Koné - nearest hotel: Hotel Santa Eliza
Sinabang - nearest hotel: Hotel Santa Eliza
Gordonvale - nearest hotel: Hotel Santa Eliza
Al Muwayh - nearest hotel: Hotel Santa Eliza
Boyolangu - nearest hotel: Hotel Santa Eliza
Tabou - nearest hotel: Hotel Santa Eliza
Kapaa - nearest hotel: Hotel Santa Eliza
Yangon - nearest hotel: Hotel Santa Eliza
Marabu - nearest hotel: Hotel Santa Eliza
Simanggang - nearest hotel: Hotel Santa Eliza
Toliara - nearest hotel: Hotel Santa Eliza
Pisco - nearest hotel: Hotel Santa Eliza
Haiku-Pauwela - nearest hotel: Hotel Santa Eliza
Angoche - nearest hotel: Hotel Santa Eliza
Alice Springs - nearest hotel: Hotel Santa Eliza

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Kourou,GF,5.1552,-52.6478,92,Hotel Santa Eliza
1,Wailua Homesteads,US,22.0669,-159.3780,80,Hotel Santa Eliza
2,Dawei,MM,14.0833,98.2000,79,Hotel Santa Eliza
3,Jwaneng,BW,-24.6004,24.7303,18,Hotel Santa Eliza
4,Cabo San Lucas,MX,22.8909,-109.9124,61,Hotel Santa Eliza
...,...,...,...,...,...,...
88,Manokwari,ID,-0.8667,134.0833,84,Hotel Santa Eliza
89,Gorontalo,ID,0.5412,123.0595,90,Hotel Santa Eliza
90,Capitán Bado,PY,-23.2667,-55.5333,84,Hotel Santa Eliza
91,Biltine,TD,14.5333,20.9167,15,Hotel Santa Eliza


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    c="Hotel Name",
    color = "City",  
    colorbar=True,
    size=20,
    hover_cols=["Hotel Name", "City", "Country"]
)

# Display the map plot
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)